<a href="https://colab.research.google.com/github/mihaidobri/Natural_Disasters_Tweets_Analysis/blob/master/tweet_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## lstm



https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/


Glove: https://nlp.stanford.edu/projects/glove/


[Keras Tokenizer](https://keras.io/preprocessing/text/)




In [0]:
from IPython.display import clear_output
from google.colab import drive

drive.mount('/gdrive')
clear_output()

%cd /gdrive/My\ Drive

!ls

In [0]:
# used to clean console
from IPython.display import clear_output

!wget https://www.dropbox.com/s/z7iv4o9bxtll3w1/sandy_glove_token.csv
clear_output()

!wget http://nlp.stanford.edu/data/glove.6B.zip
clear_output()

!unzip glove.6B.zip
clear_output()

!ls

glove.6B.zip  sample_data  sandy_glove_token.csv


In [0]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import string
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt


Using TensorFlow backend.


### Read data

In [0]:
# read file
df = pd.read_csv("sandy_glove_token.csv", usecols=["tweet id", " tweet", " label", "tweet_cleaned"], index_col=None)

# print sample
print(df["tweet_cleaned"][0])
print(df[" tweet"][0])
print(df[" label"][0])


i 've got enough candles to supply a mexican family
I've got enough candles to supply a Mexican family
off-topic


### Output of model: on-topic/off-topic -> 1/0

In [0]:
# Output labels
y = df[" label"]
print(y[0])

# encoding 0-1
y = [1 if label=="on-topic" else 0 for label in y]
print(y[0])

off-topic
0


### Input of model

In [0]:
# Input
X = (df["tweet_cleaned"])

print("Number of Tweets: ",len(X))

Number of Tweets:  10008


### Split to Train Data and Test Data

In [0]:
# split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("Train size:\t",len(X_train))
print("Test size:\t", len(X_test))

Train size:	 8006
Test size:	 2002


### Variables for embeddings

In [0]:
# variables 
top_words = 2000 # most frequet words to use from vocab
max_words = max([len(tweet) for tweet in X])    # max number of tweet
emb_size = 100    # embedding size
emb_file = 'glove.6B.%sd.txt'%(emb_size)   # emb file

### Tokenizer that translate text to numbers tk

In [0]:
# tokenizing tweet
tk = Tokenizer(num_words=(top_words), filters=string.punctuation, lower=True, split=' ', char_level=False, oov_token='<unk>')

# fit tokenizer
for i,tweet in enumerate(X_train):
  # feedback of loop - progress
  if i%1000 == 0:
    print("Progress %s of %s"%(i,len(X_train)))
  tk.fit_on_texts([tweet])

## **Key Step**
tk.word_index = {e:i for e,i in tk.word_index.items() if i <= top_words} # <= because tokenizer is 1 indexed
tk.word_index[tk.oov_token] = top_words

Progress 0 of 8006
Progress 1000 of 8006
Progress 2000 of 8006
Progress 3000 of 8006
Progress 4000 of 8006
Progress 5000 of 8006
Progress 6000 of 8006
Progress 7000 of 8006
Progress 8000 of 8006


In [0]:
tk.texts_to_sequences(["Eu is from the Sinaia"])

[[2000, 14, 55, 5, 2000]]

### Transform Text Data to sequence of numbers using tk (tokenizer)

### Padding to make all tweets same size as max_words

In [0]:
X_train = tk.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_words, padding='post')


X_test = tk.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_words, padding='post')

### Sample data of text transformed to sequence of numbers

In [0]:
# sample example
print(df["tweet_cleaned"][11])
print(tk.texts_to_sequences([df["tweet_cleaned"][11]]))


<user> need me to comeback and finish it since i was supposed to sleep over and then i woulda finished
[[3, 107, 31, 9, 2000, 18, 1311, 24, 252, 6, 46, 432, 9, 311, 96, 18, 196, 6, 2000, 1877]]


### Load Glove Embeddings - creates dictionary word-embedding pair

In [0]:
# load embedding file
embeddings_index = dict()

# read embedding file
with open(emb_file, 'r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    emb = np.array(values[1:], dtype='float32')
    embeddings_index[word] = emb
    

print('Loaded %s word vectors.' % len(embeddings_index))

FileNotFoundError: ignored

### Embedding Matrix

In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((top_words+1, emb_size))

vocabulary = tk.word_index

for word, i in vocabulary.items():
  if i == top_words+1:
    break  
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

### Create LSTM Model

In [0]:
model = Sequential()
model.add(Embedding(input_dim=top_words+1, output_dim=emb_size, weights=[embedding_matrix], input_length=max_words, trainable=False))


"""CNN""" 
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())


"""CNN + LSTM with dropout"""
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add((LSTM(256, dropout=0.2, recurrent_dropout=0.2)))



"""LSTM with dropout"""
# model.add((LSTM(256, dropout=0.2, recurrent_dropout=0.2)))


model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

### Training

In [0]:
history = model.fit(X_train, y_train,  validation_split=0.15, epochs=20, batch_size=50)

### Prediction

In [0]:
y_pred = model.predict(X_test)

# round predictions
y_pred = [round(x[0]) for x in y_pred]

### Evaluation

In [0]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

### Plot

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Naive Bayes


## import libraries

In [0]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from keras.preprocessing.text import Tokenizer
import string
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

## Read CSV data

In [0]:
# read file
df = pd.read_csv("sandy_glove_token.csv", usecols=["tweet id", " tweet", " label", "tweet_cleaned"], index_col=None)

# print sample
print(df["tweet_cleaned"][0])
print(df[" tweet"][0])
print(df[" label"][0])


## Get label output data and encode it

In [0]:
# Output labels
y = df[" label"]
print(y[0])

# encoding 0-1
y = [1 if label=="on-topic" else 0 for label in y]
print(y[0])

## Get input data

In [0]:
# Input
X = (df["tweet_cleaned"])

print("Number of Tweets: ",len(X))

## Split data into Train - Test

In [0]:
# split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("Train size:\t",len(X_train))
print("Test size:\t", len(X_test))

## Get Maximum tweet length

In [0]:
# variables 
max_words = max([len(tweet) for tweet in X])    # max number of tweet

## Fit Tokenizer

In [0]:
# tokenizing tweet
tk = Tokenizer(filters=string.punctuation, lower=True, split=' ', char_level=False, oov_token='<unk>')

# fit tokenizer
for i,tweet in enumerate(X_train):
  # feedback of loop - progress
  if i%1000 == 0:
    print("Progress %s of %s"%(i,len(X_train)))
  tk.fit_on_texts([tweet])

## **Key Step**
tk.word_index = {e:i for e,i in tk.word_index.items() if i <= top_words} # <= because tokenizer is 1 indexed
tk.word_index[tk.oov_token] = top_words

## Encode training data

In [0]:
X_train = tk.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_words, padding='post')


X_test = tk.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_words, padding='post')

## Print Sample code

In [0]:
# sample example
print(df["tweet_cleaned"][11])
print(tk.texts_to_sequences([df["tweet_cleaned"][11]]))


# Create Gaussian NB and Train

In [0]:
clf = GaussianNB()

clf.fit(X_train, y_train)

## User Gaussian NB to make predictions

In [0]:
y_pred = clf.predict(X_test)

 # Evaluation On Test

In [0]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)